<a href="https://colab.research.google.com/github/zanzivyr/Optimizers/blob/main/QP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quadratic Programming

Today we will continue our study of optimization by writing a Quadratic Programming solver.

This will be a bit more challenging than the LP Solver, but will help build understanding of different classes of optimization problems. Let's dig in!

# Resources

These are the resources I used to learn how to solve Quadratic Programming problems.

- Wikipedia - https://en.wikipedia.org/wiki/Quadratic_programming
- Kody Powell - https://www.youtube.com/watch?v=GZb9647X8sg
- Chris Rycroft - https://www.youtube.com/watch?v=O-pTuBTShc4
- BYU FLOW Lab - https://youtu.be/F9aG69nn1n8
- **BEST** -> Efstratios Nikolaidis - https://www.youtube.com/watch?v=Ifve4IpeH9w

Automatic Differentiation:
- Ari Seff - https://www.youtube.com/watch?v=wG_nF1awSSY


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from numpy import linalg as LA

In [18]:
constraints = 4 # m
variables = 2 # n

# Q must be Symmetric Positive Definite
# Q is n x n
Q = tf.random.normal(
    shape=[variables, variables],
    mean=0,
    stddev=1,
    dtype=tf.dtypes.float32,
    seed=42,
    name=None
)
# f is n x 1
f = tf.random.normal(
    shape=[variables, 1],
    mean=0,
    stddev=1,
    dtype=tf.dtypes.float32,
    seed=32,
    name=None
)
# A is m x n
A = tf.random.normal(
    shape=[constraints, variables],
    mean=0,
    stddev=1,
    dtype=tf.dtypes.float32,
    seed=22,
    name=None
)
# b is m x 1
b = tf.random.normal(
    shape=[constraints, 1],
    mean=0,
    stddev=1,
    dtype=tf.dtypes.float32,
    seed=12,
    name=None
)
# solution is (m + n) x 1
sol = np.concatenate([-f, -b], 0)

Q, f, A, b, sol.shape

(<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[-1.4677944 ,  1.0870852 ],
        [ 0.94264466, -0.40158314]], dtype=float32)>,
 <tf.Tensor: shape=(2, 1), dtype=float32, numpy=
 array([[2.082934],
        [1.77611 ]], dtype=float32)>,
 <tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[ 0.16568154, -3.0080743 ],
        [-0.6044153 , -0.10841811],
        [-0.20918587, -0.71651995],
        [ 0.16861683,  1.3579242 ]], dtype=float32)>,
 <tf.Tensor: shape=(4, 1), dtype=float32, numpy=
 array([[-1.0506643 ],
        [ 0.6807904 ],
        [-0.11149635],
        [ 0.06687231]], dtype=float32)>,
 (6, 1))

In [19]:
R1 = tf.concat([Q, A], 0)

# Create right side of Jacobian matrix
# Add appropriate number of rows of zeroes
R2 = A
if A.shape[1] != R1.shape[0]:
  R2 = tf.concat([A, tf.zeros([constraints, A.shape[0]], tf.float32)], 1)

R = tf.concat([R1, np.transpose(R2)], 1)
R.shape

TensorShape([6, 6])

# Singular Value Decomposition

We will use SVD to decompose and invert our R matrix containing the Hessian of the Lagrangian, A, and A transpose.

In [29]:
# SVD
u, s, vt = np.linalg.svd(R, full_matrices=True)
u.shape, s.shape, vt.shape

((6, 6), (6,), (6, 6))

In [30]:
# Moore-Penrose Pseudo Inverse
vtt = np.transpose(vt)
s = np.linalg.inv(np.diag(s))
ut = np.transpose(u)

vtt.shape, s.shape, ut.shape

((6, 6), (6, 6), (6, 6))

## Reshaping

Let's reshape the sigma matrix.

If it's overdetermined (rows > cols), then add a column of zeroes.

If it's underdetermined (cols > rows), then add a row of zeroes.

In [31]:
# Overdetermined
# Concat extra column of zeroes
if R.shape[0] > R.shape[1]:
  print("Overdetermined")
  sinv = np.concatenate(
      (
          s, 
          tf.zeros([s.shape[0], 1], tf.float32)
      ), 
      axis=1
  )
# Underdetermined
# Concat extra row of zeroes
elif R.shape[0] < R.shape[1]:
  print("Underdetermined")
  sinv = np.concatenate(
      (
          s, 
          tf.zeros([1, s.shape[1]], tf.float32)
      ), 
      axis=0
  )
else:
  sinv = s

sinv.shape

(6, 6)

Now, given the linear affine equation, solve for the step column vector.

In [25]:
Adag = np.dot(vtt, np.dot(sinv, ut))
step = np.dot(Adag, sol)
step

array([[ 1.1524588e+00],
       [ 3.4936219e-03],
       [-1.5648152e+15],
       [ 6.5487091e+14],
       [-1.0237997e+16],
       [-8.8162579e+15]], dtype=float32)

## Solution

Assuming that Q is Symmetric and Positive Definite, and the Hessian of the Lagrangian contains only linear terms, then we can stop here.

Below is our solution for x given the Lagrange multiplier, lambda.

In [39]:
sx = step[0:variables,:]
sx

array([[1.1524588 ],
       [0.00349362]], dtype=float32)

In [40]:
slmda = step[variables:,:]
slmda

array([[-1.5648152e+15],
       [ 6.5487091e+14],
       [-1.0237997e+16],
       [-8.8162579e+15]], dtype=float32)

# Non-Linear Hessians

If the Hessian of the Lagranian contains non-linear terms then we will not be able to differentiate out our decision variables. 

This means that step values for x and lambda will need to be substituted back into the overall matrix, R (Rs = d, where R = [ [Q Aᵀ], [A 0] ], s = [x, λ], and d = [-f, -b]), before solving again for s and continuing to step towards the optimum.

I believe this requires using Automatic Differentiation, which is a topic wholly different from Quadratic Programming, and I will leave for another project.

In [37]:
iter = 0 # Need to implement Automatic Differentiation
thresh = 0.01

lmda = 0

J = 0
Jprev = 0
L2 = 0
L2prev = 0

# initial guess
x = tf.random.normal(
    shape=[variables, 1],
    mean=0,
    stddev=1,
    dtype=tf.dtypes.float32,
    seed=2,
    name=None
)

for i in range(iter):
  J = np.dot(np.transpose(x), np.dot(Q, x)) + np.dot(np.transpose(f), x)
  L2 = LA.norm(sx, axis=0)

  slope = (J - Jprev) / (L2 - L2prev)
  
  Jprev = J
  L2prev = L2

  # Need to reduce Hessian of Lagrangian using previous step 
  #  values for x and lambda to find new step value for x and lambda.
  
  # If the Hessian contains non-linear terms, then I believe this 
  #  cannot be done without Automatic Differentiation.

  # We will leave this as a task for another project.
  x = x + sx
  lmda = lmda + slmda

  if slope < thresh:
    print("Found optimal value!")
    break

'''
print("Optimal cost is: " + str(J))
print("with x as: ")
print(x)
print("and lambda as: ")
print(lmda)
'''

'\nprint("Optimal cost is: " + str(J))\nprint("with x as: ")\nprint(x)\nprint("and lambda as: ")\nprint(lmda)\n'